# Combine results of simulations and make bar plots

Use plotly to display results from a list of supernovae model simulations processed with sspike.

## Create event totals data frame from list of models

Each processed model should have a `totals.txt` file with tabulated event rates by channel.

In [41]:
from os import listdir
from os.path import isfile

import pandas as pd

from sspike.targets import Target
from sspike.beer import sort_channels

### Get file paths to tabulated results

In [18]:
# Path to sspike snowball directory.
snowball_dir = '/Users/joe/src/gitjoe/sspike/snowballs/'

# List simulations of interest.
sn_names = ['F21-20', 'N13-20-20-300', 'W20-20.0-1.25']
distance = 5

# List for filepaths to totals files tabulated using sspike.beer.tab()
beer_tabs = []

# Folder containing simulation results for each model.
for name in sn_names:
    sn_dir = f"{snowball_dir}{name}/"
    # Results for distance of interest.
    for d in listdir(sn_dir):
        if f"{distance}kpc" in d:
            file_name = f"{sn_dir}{d}/totals.txt"
            if isfile(file_name):
                beer_tabs.append(file_name)
            else:
                print(f'File not found!\n{file_name}')

### Set target for channel lists.

In [54]:
target = Target('kamland')
combos = {**sort_channels(target.snow_channels),
          **sort_channels(target.nc_channels)}

### Read each file into a data frame

In [62]:
# Columns for simulation type and events per channel.
# TODO: add other simulation properties for sorting.
column_names = ['sim'] + list(combos.keys())
events = pd.DataFrame()
for i, sn in enumerate(sn_names):
    # One row for each simulation file.
    row = {'sim': sn}
    for channel in combos:
        row[channel] = 0.

    with open(beer_tabs[i], 'r') as f:
        lines = f.readlines()

    # Sort by data type/processing method.
    skip = 0
    data_type = None
    nc_flav_count = 0
    for line in lines:
        # Skip blank line separators.
        if not line.strip():
            continue

        # Skip lines that are for checking, but not really of interest here.
        if skip:
            skip -= 1
            continue

        # Data types of interest: snow-smeared, sspike-nc.
        if len(line.split('-')) == 2:
            data_type = line.strip()

            # TODO: make this check target channel lengths.
            if data_type == 'snow-smeared':
                skip = 1
            if data_type == 'snow-unsmeared':
                skip = 23
            if data_type == 'sspike-basic':
                skip = 5
            if data_type == 'sspike-nc':
                skip = 21
            continue
        
        try:
            channel, count = line.split(':')
        except:
            print(line)
            continue
        for column in combos.keys():
            if channel in combos[column]:
                row[column] += float(count)
                if '_p' in column:
                    nc_flav_count += 1
                break
        if nc_flav_count == 4:
            break
    events = events.append(row, ignore_index=True)

In [63]:
events

,sim,ibd,e,C12-nue,C12-nuebar,C12-nc,C13-nue,C13-nc,nc_nue_p,nc_nuebar_p,nc_nux_p,nc_nuxbar_p
0,F21-20,1487.111905,79.015312,106.223030,121.580308,243.472881,2.419639,6.459978,30.245743,44.103530,34.660981,34.660981
1,N13-20-20-300,325.441848,31.038321,4.146606,8.474103,41.643998,0.529867,1.824775,0.222564,0.476586,4.203557,4.203557
2,W20-20.0-1.25,466.692736,33.280216,4.016982,13.297901,60.889099,0.665346,2.312129,0.004581,0.050648,1.203849,1.203849
